![](https://raw.githubusercontent.com/stack-of-tasks/pinocchio/master/doc/images/pinocchio-logo-large.png)

# Pinocchio: rigib-body derivatives


In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Credits
More than anywhere in Pinocchio, the derivative algorithms are the results of the hard work of Justin Carpentier. Read more about the mathematics behing the code in *Carpentier and Mansard, "Analytical derivatives of rigid body dynamics algorithms", RSS 2018*.

## Set up
We will need Pinocchio, the robot models stored in the package *example-robot-data*, a viewer (either GepettoViewer or MeshCat), some basic linear-algebra operators and the SciPy optimizers.

In [26]:
import pinocchio as pin
import example_robot_data as robex
import numpy as np
from numpy.linalg import inv,pinv,eig,norm,svd,det
from scipy.optimize import fmin_bfgs
import time
import copy
np.set_printoptions(precision=2, linewidth=200, suppress=True)

## Quick description of the tutorial

We will follow the same roadmap as for the previous tutorial, and compute the derivatives of each cost function. We then re-start with a manipulator robot, that has a regular vector configuration (no fancy Lie group there in the begining), and only consider free-basis robot at the very end. 

In [27]:
robot = robex.loadTalosArm()   # Load a 6-dof manipulator arm

Viewer = pin.visualize.GepettoVisualizer
#Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.display(robot.q0)

# jupyter_cell does not like to be inside a if/then block
isinstance(viz,pin.visualize.MeshcatVisualizer) and viz.viewer.jupyter_cell()  ;

In [28]:
rmodel = robot.model
rdata = rmodel.createData()

# Arbitrarily selects a frame (and the parent joint) for later cost functions.
frameIndex = rmodel.getFrameId('gripper_left_fingertip_1_link')
jointIndex = rmodel.frames[frameIndex].parent


## 1. Derivatives of the 6d "placement" cost

We first study the derivatives of the 6d cost. As Pinocchio works with spatial "6d" quantities, this derivative is indeed slightly more intuitive that the one of the 3d cost.

**Notations**: For the derivations that follows, let denote by $\ ^oM_*$ the target placement `Mtarget` and by $\ ^oM_e(q)$ the placement of the operational frame `rdata.oMf[frameIndex]`.

Let's recall first that the 6d cost function is the log of the relative placement $cost(q) = log(^oM_{*}^{-1} \ ^oM_e(q))$, with $\ ^oM_{*}$ a fixed placement, and $\ ^oM_e(q)$ the placement of a given operational frame $\mathcal{F}_e$ of the robot. Applying [the chain rule](https://www.khanacademy.org/math/ap-calculus-ab/ab-differentiation-2-new/ab-3-1a/v/chain-rule-introduction), the derivative of $cost$ must be the product of two derivatives: the derivative of $log$, and the derivative of the relative placement.


In [48]:
# This is a copy of the code explained in the first notebook:
q = pin.randomConfiguration(rmodel)
Mtarget = pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.27])) # arbitrary values
pin.forwardKinematics(rmodel,rdata,q)
Meff = rdata.oMf[frameIndex]
targetMeff = Mtarget.inverse()*Meff
residual = pin.log(targetMeff).vector
cost = sum(residual**2)

### Derivative of the placement of a frame attached to the robot
#### Spatial velocities
The derivative of a place $\ ^AM_B(q)$ is a spatial velocity, denoted $\nu_{AB}$. The spatial velocity is the representation of the vector field of 3D velocity of each point of the rigid body. In Pinocchio, $\nu$ can be represented in two ways:
- either in the $\mathcal{F}_B$ frame, $\ ^B\nu_{AB} = (\ ^Bv_B, \ ^B\omega)$. In that case both the linear part $v$ and the angular part $\omega$ are represented in the $B$ frame, while $v_B$ is the velocity of the center of the frame $\mathcal{F}_B$.
- or in the $\mathcal{F}_A$ frame, $\ ^A\nu_{AB} = (\ ^Av_A, \ ^A\omega)$. In that case, $v$ and $\omega$ are expressed along the frames of $\mathcal{F}_A$, and $v_A$ is the velocity of the point rigidly attached to the body passing through the centre of $\mathcal{F}_A$ at this instant. 

Spatial velocities are implemented in Pinocchio by the class `pin.Motion`, and are respectively argument and output of `pin.exp` and `pin.log`.

#### Joint jacobians
When the spatial velocity of a frame attached to the robot is the consequence of a joint velocity $v_q$, it can be computed as $\nu = J(q) v_q$. As we said, $\nu$ can be represented in an arbitrary frame. The two logical choices are either the `pin.WORLD` frame, i.e. the frame attached to the universe joint `rmodel.joints[0]`; or it can be the local joint attached to the frame we are observing.

Similarly, the Jacobian should be expressed in either of these two frames $\ ^o\nu = \ ^oJ(q) v_q$ or $\ ^e\nu = \ ^eJ(q) v_q$ (where $\ ^oe_E(q)$ is  operational frame of interest).

In [31]:
pin.computeJointJacobians(rmodel,rdata,q)   ### precomputes all jacobians
oJ = pin.getJointJacobian(rmodel,rdata,jointIndex,pin.WORLD)  # in world frame
eJ = pin.getJointJacobian(rmodel,rdata,jointIndex,pin.LOCAL)  # in local frame

The shortcut `pin.computeJacobian(rmodel,rdata,q,jointIndex)` computes a single jacobian, without pre-calculation, but only in the local frame (as running this version of the algorithm in the world frame is not efficient).

#### Frame jacocians
We yet gave the syntax for evaluating the jacobian of a frame attached to a joint. The syntax is quite similar for frame jacobians.

In [33]:
pin.computeJointJacobians(rmodel,rdata,q)   ### precomputes all jacobians
oJf = pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.WORLD)  # in world frame
fJf = pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.LOCAL)

Actually, as the operational frame and the joint space are rigidly attached, their velocity vector fields are the same, hence the expression of their spatial velocity in a same frame are equals, hence their world jacobians are equal.

In [37]:
assert(norm(oJf-oJ)==0)  ### no numerical rounding errors here, they are exactly the same

#### Changing the frame of expression of velocities
If we want to expressed the spatial velocity in another frame, we can move its expression with the corresponding SE3 displacement: change $\ ^A\nu$ expressed in $\mathcal{F}_A$ into $\ ^B\nu$ expressed in $\mathcal{F}_B$ is done with the so-called SE(3) **action**. 

In [40]:
aMb = pin.SE3.Random()
anu = pin.Motion.Random()
bnu = aMb.act(anu)

The SE3 action, also call "big" Adjoint, is a linear operation in $\nu$, that we denote by the action matrix $\ ^AX_B$. The action matrix can be explicited with:

In [42]:
aXb = aMb.action
bnu_vec = aXb @ anu.vector
assert(norm(bnu_vec-bnu.vector)<1e-6)

Jacobians can be changed with the same way. Formally, the colums of the jacobian are spatial velocities, yet they are not represented that way in Pinocchio, and the `pin.SE3.act` function does not work on jacobian. You have to explicitly use the action matrix.

For example, the jacobian of the operation frame wrt the target frame, expressed in the target frame, is $\ ^*J$:

In [ ]:
targetJ = Mtarget.inverse().action @ oJ

If you followed properly, you should be convinced that the jacobian corresponding to $\ ^{target}M_{e}$ is the 
same as the one for $\ ^oM_e$.

#### Log jacobian

Now, we have two representations possible for the jacobian of $(\ ^oM_{*}^{-1} \ ^oM_e)$: either $\ ^*J$ or $\ ^eJ$. Which one should we choose. Well, actually, it depends on the choise we make for representing the jacobian of the log.

M(q) is a function that maps the configuration space (a vector space, so far) into the Lie group SE(3). On the other hand, the *log* is a function thats maps the same Lie group SE(3) into the vector space $\mathbb{R}^6$ (or, more specifically, the space of spatial velocity, which is a real vector space of dimension 6). So, similarly to the jacobian of M, the jacobian of *log* can be represented either in the frame attache to the operational frame $\mathcal{F}_e$ or to the frame attached to the fixed target $\mathcal{F}_*. 

Let's look at the documentation of `pin.Jlog6` to knwo which frame to use:

In [53]:
print(pin.Jlog6.__doc__)


Jlog6( (SE3)Spatial transform M (SE3)) -> object :
    Jacobian of log(M) which maps from the tangent of SE(3) at M to the tangent of SE(3) at Identity.


So, in clear, the frame Jacobian should be expressed in the local frame, as the jacobian of the log is also expressed in this frame.

We can now conclude.

#### Gradrient of the cost function

Our cost is $c(q) = r(q)^T r(q)$, where the residual $r(q) = log(\ ^*M_e(q))$. 
The cost gradient is then $\nabla c = \frac{d c}{d q} = 2 J^T r$, where $J = \frac{dr}{dq}$, the jacobian of the residual, is the product of the jacobian of the log $J_{log}$ and the jacobian of the operational frame placement $J_q$.
$$ \nabla c = 2 J_q^T J_{log}^T r(q)$$

 

![question](question.png)
Rewrite the cost 6d class of the previous notebook, with an additional `def calcDiff(self,q)` function that return the gradient of the cost.

In [62]:
# %load -r 42-72 costs.py
class Cost6d:
    def __init__(self,rmodel,rdata,frameIndex=None, Mtarget=None,viz=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.Mtarget = Mtarget if Mtarget is not None \
                       else pin.SE3(pin.utils.rotate('x',3.14/4), np.array([0.5, 0.1, 0.27]))  # x,y,z
        self.frameIndex = frameIndex if frameIndex is not None else robot.model.nframes-1
        self.viz = viz

    def residual(self,q):
        '''Compute score from a configuration'''
        pin.forwardKinematics(self.rmodel,self.rdata,q)
        M = pin.updateFramePlacement(self.rmodel,self.rdata,self.frameIndex)
        self.deltaM = self.Mtarget.inverse() * M
        return pin.log(self.deltaM).vector

    def calc(self,q):
        return sum(self.residual(q)**2)

    def callback(self,q):
        if self.viz is None: return
        self.viz.applyConfiguration('world/blue', pin.SE3ToXYZQUATtuple(M))
        self.viz.applyConfiguration('world/box', pin.SE3ToXYZQUATtuple(self.Mtarget))
        self.viz.display(q)
        time.sleep(1e-2)

    def calcDiff(self,q):
        J = pin.computeFrameJacobian(self.rmodel,self.rdata,q,self.frameIndex)
        r = self.residual(q)
        Jlog = pin.Jlog6(self.deltaM)
        return 2 * J.T @ Jlog.T @ r

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>pin.computeJointJacobians(rmodel,rdata,q)</code> precomputes all the jacobians.</li>
        <li><code>pin.getJointJacobian(rmodel,rdata,jointIndex,pin.ReferenceFrame)</code> and <code>pin.getFrameJacobian(rmodel,rdata,frameIndex,pin.ReferenceFrame)</code> returns the joint and frame jacobians, where <code>pin.ReferenceFrame</code> can be either <code>pin.LOCAL</code> or <code>pin.WORLD</code>. </li>
        <li><code>pin.SE3.act</code> can change the expression frame of spatial velocities.</li>
        <li><code>pin.SE3.action</code> is the $6\times 6$ action matrix, that can right multiplies joint and frame jacobians.</li>
        <li><code>pin.Jlog6</code> compute the jacobian of the log of SE(3). It is expressed</li>

     </ul>
</div>

## 2. Finite differencing for validation

... introducing the paragraph

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code></code> </li>
        <li><code></code> </li>
        <li><code></code> </li>
     </ul>
</div>

## 3. Derivatives the 3d "position" cost

... introducing the paragraph

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
     <li>Cheese</li>
     <li>Carrot</li>
     <li>Coconut</li>
     </ul>
</div>

## 4. Derivatives of the posture cost

... introducing the paragraph

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
     <li>Cheese</li>
     <li>Carrot</li>
     <li>Coconut</li>
     </ul>
</div>

## 5. Derivatives of the two gravity costs

... introducing the paragraph

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
     <li>Cheese</li>
     <li>Carrot</li>
     <li>Coconut</li>
     </ul>
</div>

## 6. The return of the free flyer

... introducing the paragraph

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
     <li>Cheese</li>
     <li>Carrot</li>
     <li>Coconut</li>
     </ul>
</div>